# Recuperation de la page

On recupere la page via le beautifulsoup


In [166]:
import requests
import json
from bs4 import BeautifulSoup

In [168]:
import pprint

In [175]:
class Content:
    def __init__(self, url, name,date_creation, projets,debut,fin,resume,strategie,sites,params):
        self.url = url
        self.datasetName = name
        self.date_creation = date_creation
        self.projets = projets
        self.date_debut = debut
        self.date_fin = fin
        self.resume = resume
        self.strategie = strategie
        self.sites = sites
        self.params = params
    def __str__(self):
        value =  f'''url: {self.url},
         name: {self.datasetName},
         date_creation: {self.date_creation},
         projets: {self.projets},
         date_debut: {self.date_debut},
         date_fin: {self.date_fin},
         resumé: {self.resume},
         strategie: {self.strategie}
        '''
        sts = ''
        i=1
        for site in self.sites:
            sts = sts + f'site {i}: {str(site)} \n'
            i=i+1
        prs = ''
        i=1
        for param in self.params:
            prs = prs + f'param {i}: {str(param)} \n'
            i=i+1
        value = value + '\n' + sts + '\n'+prs
        return value.format(self=self)
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4)
class Fiche:
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4)

class Site:
    def __init__(self,name,type_, long_min,long_max,lat_min,lat_max,alt_min,alt_max):
        self.name = name
        self.type = type_
        self.long_min = long_min
        self.long_max = long_max
        self.lat_min = lat_min
        self.lat_max = lat_max
        self.alt_min = alt_min
        self.alt_max = alt_max
    
    def __str__(self):
        value =  f'''name: {self.name},
         type: {self.type},
         longitude minimale: {self.long_min},
         longitude maximale: {self.long_max},
         latitude minimale: {self.lat_min},
         latitude maximale: {self.lat_max},
         altitude minimale: {self.alt_min},
         altitude maximale: {self.alt_max}
        '''
        return value
        
class Parametre:
    def __init__(self,nom,keywords,unite,date_deb, date_fin, precision):
        self.nom = nom
        self.keywords = keywords
        self.unite = unite
        self.date_deb = date_deb
        self.date_fin = date_fin
        self.precision = precision
    def __str__(self):
        value =  f'''nom: {self.nom},
         keywords: {self.keywords},
         unite: {self.unite},
         date debut: {self.date_deb},
         date fin: {self.date_fin},
         precision: {self.precision},
        '''
        return value

def getPage(url):
    req = requests.get(url)
    return BeautifulSoup(req.text, 'html.parser')

In [176]:
def scrapeBrookings2(url):
    bs = getPage(url)
    body = bs.find('div', {'class', 'main-content'})
    body = body.table.find_all('td')
    
    title = body[1].text
    date_creation = body[3].text
    projets = body[5].text
    debut = body[7].text
    fin = body[9].text
    resume = body[17].text
    strategie = body[19].text
    
    i = 22
    sites = []
    while 'Site ' in body[i].text:
        sites.append(Site(body[i+2].text, body[i+4].text,body[i+6].text,body[i+8].text,body[i+10].text,body[i+12].text,body[i+14].text,body[i+16].text))
        i = i+17
    params = []
    while 'Paramètre mesuré ' in body[i].text:
        params.append(Parametre(body[i+2].text,body[i+4].text,body[i+6].text,body[i+8].text,body[i+10].text,body[i+12].text))
        i = i+13
    
    return Content(url, title, date_creation,projets,debut,fin,resume,strategie,sites,params)

In [177]:
value = scrapeBrookings2("https://baobab.sedoo.fr/extract/download.php?datsId=1266")

In [178]:
print(value)

url: https://baobab.sedoo.fr/extract/download.php?datsId=1266,
         name: ACAR - US aircraft reports, GTS data,
         date_creation: 2006-06-14,
         projets: OPERATIONAL-DATA > GTS data,
         date_debut: 1999-11-04,
         date_fin: 2010-12-31,
         resumé: à compléter,
         strategie: à compléter
        
site 1: name: ACAR,
         type: AIRCRAFT,
         longitude minimale: -50,
         longitude maximale: 50,
         latitude minimale: 40,
         latitude maximale: -20,
         altitude minimale: -999,
         altitude maximale: 20026
         

param 1: nom: Air Temperature,
         keywords: Atmosphere > Atmospheric Temperature > Surface Temperature > Air Temperature,
         unite: Degrees Celsius - °C,
         date debut: 1999-11-04,
         date fin: 2010-12-31,
         precision:  ,
         
param 2: nom: Wind Speed,
         keywords: Atmosphere > Atmospheric Winds > Wind Speed,
         unite: meters per second - m/s,
         date de

In [179]:
test = value.toJSON()

In [180]:
test

'{\n    "datasetName": "ACAR - US aircraft reports, GTS data",\n    "date_creation": "2006-06-14",\n    "date_debut": "1999-11-04",\n    "date_fin": "2010-12-31",\n    "params": [\n        {\n            "date_deb": "1999-11-04",\n            "date_fin": "2010-12-31",\n            "keywords": "Atmosphere > Atmospheric Temperature > Surface Temperature > Air Temperature",\n            "nom": "Air Temperature",\n            "precision": " ",\n            "unite": "Degrees Celsius - \\u00b0C"\n        },\n        {\n            "date_deb": "1999-11-04",\n            "date_fin": "2010-12-31",\n            "keywords": "Atmosphere > Atmospheric Winds > Wind Speed",\n            "nom": "Wind Speed",\n            "precision": " ",\n            "unite": "meters per second - m/s"\n        },\n        {\n            "date_deb": "1999-11-04",\n            "date_fin": "2010-12-31",\n            "keywords": "Atmosphere > Atmospheric Pressure",\n            "nom": "Air Pressure",\n            "precis